# Detecting bad customer reviews with NLP

In [3]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
reviews_df = pd.read_csv('Hotel_Reviews.csv')

In [5]:
reviews_df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [6]:
reviews_df.shape

(515738, 17)

In [7]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [8]:
#We will combine both the columns i.e. Positive and Negative feedbacks
reviews_df["review"] = reviews_df['Positive_Review'] + reviews_df['Negative_Review']

In [9]:
reviews_df.shape

(515738, 18)

In [10]:
reviews_df['review']

0          Only the park outside of the hotel was beauti...
1          No real complaints the hotel was great great ...
2          Location was good and staff were ok It is cut...
3          Great location in nice surroundings the bar a...
4          Amazing location and building Romantic settin...
                                ...                        
515733     location no trolly or staff to help you take ...
515734     Breakfast was ok and we got earlier check in ...
515735    No Positive The ac was useless It was a hot we...
515736     The rooms are enormous and really comfortable...
515737     staff was very kind  I was in 3rd floor It di...
Name: review, Length: 515738, dtype: object

In [11]:
reviews_df['is_bad_review'] = reviews_df["Reviewer_Score"].apply(lambda x:1 if x<5 else 0)

In [12]:
reviews_df

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,review,is_bad_review
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Only the park outside of the hotel was beauti...,1
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,No real complaints the hotel was great great ...,0
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Location was good and staff were ok It is cut...,0
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Great location in nice surroundings the bar a...,1
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Amazing location and building Romantic settin...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677,location no trolly or staff to help you take ...,0
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677,Breakfast was ok and we got earlier check in ...,0
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677,No Positive The ac was useless It was a hot we...,1
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677,The rooms are enormous and really comfortable...,0


In [13]:
#feature selection:
reviews_df = reviews_df[['review','is_bad_review']]

In [14]:
reviews_df.head()

,review,is_bad_review
0,Only the park outside of the hotel was beauti...,1
1,No real complaints the hotel was great great ...,0
2,Location was good and staff were ok It is cut...,0
3,Great location in nice surroundings the bar a...,1
4,Amazing location and building Romantic settin...,0


In [15]:
reviews_df.isna().sum()

review           0
is_bad_review    0
dtype: int64

In [16]:
reviews_df

,review,is_bad_review
0,Only the park outside of the hotel was beauti...,1
1,No real complaints the hotel was great great ...,0
2,Location was good and staff were ok It is cut...,0
3,Great location in nice surroundings the bar a...,1
4,Amazing location and building Romantic settin...,0
...,...,...
515733,location no trolly or staff to help you take ...,0
515734,Breakfast was ok and we got earlier check in ...,0
515735,No Positive The ac was useless It was a hot we...,1
515736,The rooms are enormous and really comfortable...,0


In [17]:
reviews_df['review'] = reviews_df["review"].apply(lambda x: x.replace("No Negative","").replace("No Positive",""))

C:\Users\adesh\AppData\Local\Temp\ipykernel_9364\2779163841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['review'] = reviews_df["review"].apply(lambda x: x.replace("No Negative","").replace("No Positive",""))


In [18]:
reviews_df

,review,is_bad_review
0,Only the park outside of the hotel was beauti...,1
1,No real complaints the hotel was great great ...,0
2,Location was good and staff were ok It is cut...,0
3,Great location in nice surroundings the bar a...,1
4,Amazing location and building Romantic settin...,0
...,...,...
515733,location no trolly or staff to help you take ...,0
515734,Breakfast was ok and we got earlier check in ...,0
515735,The ac was useless It was a hot week in vienn...,1
515736,The rooms are enormous and really comfortable...,0


In [19]:
#nltk
!pip install nltk

from nltk.corpus import wordnet

#pos tagging
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [21]:
def clean_text(text):
    text = text.lower()   # to convert text into lower case
    text = [word.strip(string.punctuation) for word in text.split(" ")]  # tokenization and removal of punctuations
    text = [word for word in text if not any(c.isdigit() for c in word)]    # to remove words that contain numbers
    
    #stop removal
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    
    #remove empty tokens
    text = [t for t in text if len(t)>0]
    
    #pos tags
    pos_tags = pos_tag(text)
    
    #lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    
    #remove words with only one letter
    text = [t for t in text if len(t)>1]
    text = " ".join(text)
    return(text)

In [22]:
# clean text data
reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))

C:\Users\adesh\AppData\Local\Temp\ipykernel_9364\1179833129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))


In [ ]:
# Process the DataFrame in smaller batches
batch_size = 1000  # Adjust batch size based on your system's memory
for i in range(0, len(reviews_df), batch_size):
    reviews_df.loc[i:i+batch_size-1, "review_clean"] = reviews_df.loc[i:i+batch_size-1, "review"].apply(clean_text)


In [ ]:
reviews_df["review_clean"]

0         park outside hotel beautiful angry make post a...
1         real complaint hotel great great location surr...
2         location good staff ok cute hotel breakfast ra...
3         great location nice surroundings bar restauran...
4         amaze location building romantic setting book ...
                                ...                        
515733         location trolly staff help take luggage room
515734    breakfast ok get earlier check hotel look like...
515735              ac useless hot week vienna give hot air
515736    room enormous really comfortable believe famil...
515737                      staff kind floor work free wife
Name: review_clean, Length: 515738, dtype: object

In [ ]:
#add sentiment 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews_df['sentiments'] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))

C:\Users\adesh\AppData\Local\Temp\ipykernel_11348\2582203548.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['sentiments'] = reviews_df["review"].apply(lambda x: sid.polarity_scores(x))


In [ ]:
reviews_df

,review,is_bad_review,review_clean,sentiments
0,Only the park outside of the hotel was beauti...,1,park outside hotel beautiful angry make post a...,"{'neg': 0.084, 'neu': 0.865, 'pos': 0.051, 'co..."
1,No real complaints the hotel was great great ...,0,real complaint hotel great great location surr...,"{'neg': 0.058, 'neu': 0.757, 'pos': 0.186, 'co..."
2,Location was good and staff were ok It is cut...,0,location good staff ok cute hotel breakfast ra...,"{'neg': 0.122, 'neu': 0.74, 'pos': 0.138, 'com..."
3,Great location in nice surroundings the bar a...,1,great location nice surroundings bar restauran...,"{'neg': 0.103, 'neu': 0.783, 'pos': 0.114, 'co..."
4,Amazing location and building Romantic settin...,0,amaze location building romantic setting book ...,"{'neg': 0.058, 'neu': 0.861, 'pos': 0.081, 'co..."
...,...,...,...,...
515733,location no trolly or staff to help you take ...,0,location trolly staff help take luggage room,"{'neg': 0.13, 'neu': 0.71, 'pos': 0.16, 'compo..."
515734,Breakfast was ok and we got earlier check in ...,0,breakfast ok get earlier check hotel look like...,"{'neg': 0.0, 'neu': 0.644, 'pos': 0.356, 'comp..."
515735,The ac was useless It was a hot week in vienn...,1,ac useless hot week vienna give hot air,"{'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'comp..."
515736,The rooms are enormous and really comfortable...,0,room enormous really comfortable believe famil...,"{'neg': 0.0, 'neu': 0.841, 'pos': 0.159, 'comp..."


In [ ]:
reviews_df = pd.concat([reviews_df.drop(['sentiments'],axis=1), reviews_df['sentiments'].apply(pd.Series)],axis=1)

In [ ]:
reviews_df

,review,is_bad_review,review_clean,neg,neu,pos,compound
0,Only the park outside of the hotel was beauti...,1,park outside hotel beautiful angry make post a...,0.084,0.865,0.051,-0.9245
1,No real complaints the hotel was great great ...,0,real complaint hotel great great location surr...,0.058,0.757,0.186,0.9494
2,Location was good and staff were ok It is cut...,0,location good staff ok cute hotel breakfast ra...,0.122,0.740,0.138,-0.2263
3,Great location in nice surroundings the bar a...,1,great location nice surroundings bar restauran...,0.103,0.783,0.114,0.1910
4,Amazing location and building Romantic settin...,0,amaze location building romantic setting book ...,0.058,0.861,0.081,0.5263
...,...,...,...,...,...,...,...
515733,location no trolly or staff to help you take ...,0,location trolly staff help take luggage room,0.130,0.710,0.160,0.1280
515734,Breakfast was ok and we got earlier check in ...,0,breakfast ok get earlier check hotel look like...,0.000,0.644,0.356,0.7351
515735,The ac was useless It was a hot week in vienn...,1,ac useless hot week vienna give hot air,0.149,0.851,0.000,-0.4215
515736,The rooms are enormous and really comfortable...,0,room enormous really comfortable believe famil...,0.000,0.841,0.159,0.5563


In [ ]:
reviews_df["nb_chars"] = reviews_df["review"].apply(lambda x:len(x))

reviews_df["nb_words"] = reviews_df["review"].apply(lambda x: len(x.split(" "))) 

In [ ]:
reviews_df

,review,is_bad_review,review_clean,neg,neu,pos,compound,nb_chars,nb_words
0,Only the park outside of the hotel was beauti...,1,park outside hotel beautiful angry make post a...,0.084,0.865,0.051,-0.9245,1913,407
1,No real complaints the hotel was great great ...,0,real complaint hotel great great location surr...,0.058,0.757,0.186,0.9494,611,105
2,Location was good and staff were ok It is cut...,0,location good staff ok cute hotel breakfast ra...,0.122,0.740,0.138,-0.2263,301,62
3,Great location in nice surroundings the bar a...,1,great location nice surroundings bar restauran...,0.103,0.783,0.114,0.1910,1221,235
4,Amazing location and building Romantic settin...,0,amaze location building romantic setting book ...,0.058,0.861,0.081,0.5263,774,147
...,...,...,...,...,...,...,...,...,...
515733,location no trolly or staff to help you take ...,0,location trolly staff help take luggage room,0.130,0.710,0.160,0.1280,69,15
515734,Breakfast was ok and we got earlier check in ...,0,breakfast ok get earlier check hotel look like...,0.000,0.644,0.356,0.7351,87,21
515735,The ac was useless It was a hot week in vienn...,1,ac useless hot week vienna give hot air,0.149,0.851,0.000,-0.4215,77,19
515736,The rooms are enormous and really comfortable...,0,room enormous really comfortable believe famil...,0.000,0.841,0.159,0.5563,125,25


In [ ]:
# create doc2vec vector columns
!pip install gensim

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(reviews_df["review_clean"].apply(lambda x: x.split(" ")))]

# train a Doc2Vec model with our text data
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# transform each document into a vector data
doc2vec_df = reviews_df["review_clean"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
reviews_df = pd.concat([reviews_df, doc2vec_df], axis=1)


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


NameError: name 'reviews_df' is not defined

In [ ]:
reviews_df

NameError: name 'reviews_df' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF
tfidf = TfidfVectorizer(min_df=10)

# Keep the result in sparse format (do not convert to .toarray())
tfidf_result = tfidf.fit_transform(reviews_df["review_clean"])

# Create DataFrame directly from sparse matrix (it’s allowed!)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_result, columns=tfidf.get_feature_names_out())

# Optional: Rename columns to 'word_' + word
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]

# Match indexes
tfidf_df.index = reviews_df.index

# Concatenate
reviews_df = pd.concat([reviews_df, tfidf_df], axis=1)


In [ ]:
reviews_df

In [ ]:
# show is_bad_review distribution
reviews_df["is_bad_review"].value_counts(normalize = True)

In [ ]:
pip install wordcloud


In [ ]:
# wordcloud function

from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(reviews_df["review"])

In [ ]:
# highest positive sentiment reviews (with more than 5 words)
reviews_df[reviews_df["nb_words"] >= 5].sort_values("pos", ascending = False)[["review", "pos"]].head(10)


In [ ]:
# lowest negative sentiment reviews (with more than 5 words)
reviews_df[reviews_df["nb_words"] >= 5].sort_values("neg", ascending = False)[["review", "neg"]].head(10)

In [ ]:
# plot sentiment distribution for positive and negative reviews

import seaborn as sns
for x in [0, 1]:
    subset = reviews_df[reviews_df['is_bad_review'] == x]
    
    # Draw the density plot
    if x == 0:
        label = "Good reviews"
    else:
        label = "Bad reviews"
    sns.distplot(subset['compound'], hist = False, label = label)

In [ ]:
#model building:
#classifications: logistic, naive bayes,decisiontree, random forest

In [ ]:
# feature selection
label = "is_bad_review"
ignore_cols = [label, "review", "review_clean"]
features = [c for c in reviews_df.columns if c not in ignore_cols]

# split the data into train and test
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews_df[features], reviews_df[label], test_size = 0.20, random_state = 42)

In [ ]:
# # train a random forest classifier
# rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
# rf.fit(X_train, y_train)

# # show feature importance
# feature_importances_df = pd.DataFrame({"feature": features, "importance": rf.feature_importances_}).sort_values("importance", ascending = False)
# feature_importances_df.head(20)



# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD

# # Step 1: Vectorize cleaned review text
# vectorizer = TfidfVectorizer(max_features=10000)
# X_tfidf = vectorizer.fit_transform(reviews_df["review_clean"])

# # Step 2: Reduce dimensions using Truncated SVD
# svd = TruncatedSVD(n_components=300, random_state=42)
# X_reduced = svd.fit_transform(X_tfidf)



In [ ]:
# ROC curve

# from sklearn.metrics import roc_curve, auc, roc_auc_score
# import matplotlib.pyplot as plt

# y_pred = [x[1] for x in rf.predict_proba(X_test)]

# fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label = 1)

# roc_auc = auc(fpr, tpr)

# plt.figure(1, figsize = (15, 10))
# lw = 2
# plt.plot(fpr, tpr, color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()








from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# Step 1: Train/Test Split on X_reduced
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, reviews_df["is_bad_review"], test_size=0.2, random_state=42)

# Step 2: Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_red, y_train_red)

# Step 3: Predict probabilities for ROC
y_pred_prob = rf.predict_proba(X_test_red)[:, 1]  # Probability of class 1

# Step 4: Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test_red, y_pred_prob, pos_label=1)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(15, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], lw=2, linestyle='--', color='gray')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
# PR curve

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.utils.fixes import signature

average_precision = average_precision_score(y_test, y_pred)

precision, recall, _ = precision_recall_curve(y_test, y_pred)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})

plt.figure(1, figsize = (15, 10))
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

In [ ]:
import pickle

In [ ]:
with open ('model.pkl','wb') as file:
    pickle.dump(model,file)